In [3]:
import gzip
import json
import dateutil.parser
import random
import numpy as np
from collections import defaultdict
import pandas as pd

In [4]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

In [ ]:
rows = []
for u in parse("australian_users_items.json.gz"):
    for r in u["items"]:
        row = r.copy()
        row["user_id"] = u["user_id"]
        rows.append(row)

au_ui = pd.DataFrame(rows)

(np.float64(0.16130237168160533), np.float64(0.16078380246088317))

(np.float64(0.14205495973904228), np.float64(0.14286125120639293))

(np.float64(0.14181936376173232), np.float64(0.14325741713517517))

(np.float64(0.3138246287114782),
 np.float64(0.16096052587282522),
 np.float64(0.14733519202437273))

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

Skipping duplicate user/item: ('46953315', 'B00QM3CNN6')
Skipping duplicate user/item: ('31616428', 'B0026RB0G8')
Skipping duplicate user/item: ('47240912', 'B008I653SC')
Skipping duplicate user/item: ('14503091', 'B003FRMRC4')
Skipping duplicate user/item: ('38538360', 'B00HVLUR86')
Skipping duplicate user/item: ('43448024', 'B00HVLUR86')
Skipping duplicate user/item: ('51525270', 'B00HVLUR86')
Skipping duplicate user/item: ('20652160', 'B004OU2IQG')
Skipping duplicate user/item: ('10964440', 'B00HVLUR86')
Skipping duplicate user/item: ('20043677', 'B00HVLUR86')
Skipping duplicate user/item: ('44796499', 'B00HVLUSGM')
Skipping duplicate user/item: ('29066899', 'B0002CZSYO')
Skipping duplicate user/item: ('10385056', 'B004OU2IQG')
Skipping duplicate user/item: ('1658551', 'B00HVLURL8')
Skipping duplicate user/item: ('907433', 'B00N9Q2E5G')
Skipping duplicate user/item: ('39412969', 'B00HVLUR86')
Skipping duplicate user/item: ('4901688', 'B00HVLUR86')
Skipping duplicate user/item: ('234

[(0.015228426395939087, 'B00H7NFDKA'),
 (0.014492753623188406, 'B00QKVV3HC'),
 (0.014492753623188406, 'B00GXRMD7W'),
 (0.014084507042253521, 'B00H7ILRRI'),
 (0.014084507042253521, 'B0057RUMPO'),
 (0.014084507042253521, 'B000B6DTYW'),
 (0.013888888888888888, 'B00L2708TI'),
 (0.013513513513513514, 'B009Z1KKWI'),
 (0.013513513513513514, 'B000VYINCW'),
 (0.013333333333333334, 'B003F2BDZQ')]